In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def scrape_service_center_details(url):
    driver.get(url)
    
    details = {
        "업체명": None,
        "주소": None,
        "전화번호": None,
        "긴급 전화번호": None,
        "운영시간": None
    }
    
    try:

        if "busanservice" in url:
            details["업체명"] = "부산 서비스"
        else:

            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "vcard"))
            )
            details["업체명"] = driver.find_element(By.CLASS_NAME, "common-name").text
        
        details["주소"] = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "street-address"))
        ).text
        
        tel_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".tel .value"))
        )

        phone_numbers = [tel.text for tel in tel_elements[:2]]
        details["전화번호"] = phone_numbers[0] if len(phone_numbers) > 0 else None
        details["긴급 전화번호"] = phone_numbers[1] if len(phone_numbers) > 1 else None
        
        hours_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//p[strong[text()='Service Hours']]"))
        )
        hours_text = hours_element.text.replace("Service Hours\n", "").replace("\n", " | ")
        details["운영시간"] = hours_text if hours_element else None

    except Exception as e:
        print(f"Error scraping {url}: {e}")
    
    return details


driver = webdriver.Chrome()  
driver.get("https://www.tesla.com/ko_KR/findus/list/services/South%20Korea")

try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "state"))
    )

    service_centers = driver.find_elements(By.CSS_SELECTOR, ".state .anchor-container a")
    
    service_center_urls = [link.get_attribute('href') for link in service_centers]
    results = pd.DataFrame(columns=["업체명", "주소", "전화번호", "긴급 전화번호", "운영시간"])

    for url in service_center_urls:
        details = scrape_service_center_details(url)
        results = pd.concat([results, pd.DataFrame([details])], ignore_index=True)

finally:
    driver.quit()

df = results

df['주소'] = df['주소'].str.replace('특별시|광역시|특별자치도|특별자치시|<p>', '', regex=True)
df['주소'] = df['주소'].str.replace('서울시', '서울', regex=False)
df['주소'] = df['주소'].str.replace('경기도', '경기', regex=False)
df['주소'] = df['주소'].str.replace('강원도', '강원', regex=False)
df['주소'] = df['주소'].str.replace('충청북도', '충북', regex=False)
df['주소'] = df['주소'].str.replace('충청남도', '충남', regex=False)
df['주소'] = df['주소'].str.replace('전라북도', '전북', regex=False)
df['주소'] = df['주소'].str.replace('전라남도', '전남', regex=False)
df['주소'] = df['주소'].str.replace('경상북도', '경북', regex=False)
df['주소'] = df['주소'].str.replace('경상남도', '경남', regex=False)

# '시도' 열에 주소 첫 두 글자 추가
df['시도'] = df['주소'].str[:2]

# '시군구' 열에 첫 번째와 두 번째 띄어쓰기 사이의 텍스트 추출
df['시군구'] = df['주소'].str.extract(r'^\S+\s(\S+)')[0]

df['시도+시군구'] = df['시도'] + df['시군구']

results.to_excel('테슬라_최종.xlsx')